In [2]:
print("Data ingestion with dlt")

Data ingestion with dlt


In [3]:
print("Connected to GitHub")

Connected to GitHub


In [1]:
pip install dlt[duckdb]

zsh:1: no matches found: dlt[duckdb]
Note: you may need to restart the kernel to use updated packages.
